## TODO
* Features:
    * Easy visualization of: case starts, concurrent case obligations
    * printouts/reports of parameters used to generate analyses
    * utils: determine if an event falls in an arbitrary time window; aggregate to show a weekly incidence (across days of week), vs 52 weeks/yr, 12 months/year; expected formats for activity events 
    * some clinical questions to be answered: mishaps during nights and wkends vs weekdays, per provider, or likelihood when activity (anesthesia demand) is high, blocks of time with dearth of activity for didactics
    * start and end event priorities for defining an activity instance-->needs to eventually create a timeseries (long format) and use these as start and end
    
## Next
* should there be a ConcurrencyTimeSeries object to extend Series and tack on some common functionality?

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm import tqdm

sys.path.append("/home/tj/PycharmProjects/pytiva")
import pytiva

sys.path.append("/home/tj/PycharmProjects/pytiva/tests")
from tests import testconfig

def generate_testdata_study():
    testdata_csv_dict = {
        'ds_cases': { 'FILEPATH': os.path.join(testdata_wd, testconfig.TESTDATA['DS_CASES']) },
        'ds_case_meds': { 'FILEPATH': os.path.join(testdata_wd, testconfig.TESTDATA['DS_CASE_MEDS']) },
        'ds_case_events': { 'FILEPATH': os.path.join(testdata_wd, testconfig.TESTDATA['DS_CASE_EVENTS']) }
    }
    datasets = pytiva.anesthesia.datasets_from_csv_data(testdata_csv_dict)
    study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
    return study

In [ ]:
study = generate_testdata_study()
study